In [7]:
!pip install requests beautifulsoup4

In [8]:
!pip uninstall -y solana solders
!pip install solana solders

Found existing installation: solana 0.36.6
Uninstalling solana-0.36.6:
  Successfully uninstalled solana-0.36.6
Found existing installation: solders 0.26.0
Uninstalling solders-0.26.0:
  Successfully uninstalled solders-0.26.0


You can safely remove it manually.


  Using cached solana-0.36.6-py3-none-any.whl.metadata (5.3 kB)
  Using cached solders-0.26.0-cp37-abi3-win_amd64.whl.metadata (3.7 kB)
Using cached solana-0.36.6-py3-none-any.whl (62 kB)
Using cached solders-0.26.0-cp37-abi3-win_amd64.whl (5.5 MB)


In [10]:
!pip install base58

In [15]:
import requests
from bs4 import BeautifulSoup
import json
import re
from collections import defaultdict
from datetime import datetime
import time
from solana.rpc.api import Client
from solders.keypair import Keypair
from solders.transaction import Transaction
from solders.instruction import Instruction, AccountMeta
from solders.pubkey import Pubkey
from solders.compute_budget import set_compute_unit_price
import base58

# Config
RPC_ENDPOINT = "https://api.mainnet-beta.solana.com"  # Replace with private RPC for live trading
WALLET_PRIVATE_KEY = "2xYkPqWnRjT5vF8mB9cD3eHgK7uN4tQ6wA1sZ2vX9pJ8rM5nL3kP7tQ9wE4uB6yV8zC1xF2gH3jK5mN7rT9vY1bD"  # Fake key for testing
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

try:
    sol_client = Client(RPC_ENDPOINT)
    print("RPC client initialized successfully")
except Exception as e:
    print(f"Error initializing RPC client: {e}")
    sol_client = None

try:
    wallet = Keypair.from_base58_string(WALLET_PRIVATE_KEY)
    WALLET_PUBKEY = wallet.pubkey()
    print(f"Wallet loaded. Public key: {WALLET_PUBKEY}")
except Exception as e:
    print(f"Error loading wallet: {e}")
    wallet = None
    WALLET_PUBKEY = None

try:
    RAYDIUM_PROGRAM_ID = Pubkey.from_string("675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8")
    SYSTEM_PROGRAM_ID = Pubkey.from_string("11111111111111111111111111111111")
    TOKEN_PROGRAM_ID = Pubkey.from_string("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")
    print("Program IDs loaded successfully")
except Exception as e:
    print(f"Error loading program IDs: {e}")

RPC client initialized successfully
Error loading wallet: signature error: Cannot decompress Edwards point
Program IDs loaded successfully


In [17]:
def fetch_dexscreener_token_data(token_addresses):
    pairs = []
    for token in token_addresses:
        url = f"https://api.dexscreener.com/latest/dex/tokens/{token}"
        try:
            response = requests.get(url, headers=HEADERS, timeout=15)
            print(f"Token {token}: Status Code {response.status_code}")
            print(f"Raw Response: {response.text[:500]}")
            data = response.json()
            if data.get("pairs") is not None:
                sol_pairs = [pair for pair in data["pairs"] if pair["chainId"] == "solana"]
                if sol_pairs:
                    pairs.extend(sol_pairs)
                    print(f"Found {len(sol_pairs)} Solana pairs for {token}")
                else:
                    print(f"No Solana pairs found for {token}")
            else:
                print(f"No 'pairs' key in response for {token}")
        except requests.exceptions.RequestException as e:
            print(f"Network error fetching token {token}: {e}")
        except ValueError as e:
            print(f"JSON error fetching token {token}: {e}")
        except Exception as e:
            print(f"Unexpected error fetching token {token}: {e}")
        time.sleep(1)  # Avoid rate limits
    sorted_pairs = sorted(pairs, key=lambda x: float(x["volume"]["h24"]), reverse=True)
    print(f"Total Solana pairs collected: {len(sorted_pairs)}")
    return sorted_pairs[:10]

def get_social_engagement(token_name):
    return len(token_name) * 10 + hash(token_name) % 100  # Placeholder

def rank_memecoins(min_volume=100000, top_n=5):
    token_addresses = [
        "EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm",  # WIF (Dogwifhat, $50M+ volume)
        "5vYksbMzMgKTPbcbyvjo4vA7nkpG168eDcntdjuHjvkw",  # BOME (Book of Meme, $20M+ volume)
        "4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R",  # RAY (Raydium, $30M+ volume)
        "DEgTCoT3MZhRorbiXwUqT7DWfGMSnuTgoT5HdPdTXkMY",  # BILLY (Billy, $5M+ volume)
        "MEW1gQWJ3nEXg2qgERiKu7FAFj79PHvQVREQUzScpBa"   # MEW (cat in a dogs world, $3M+ volume)
    ]
    trending_pairs = fetch_dexscreener_token_data(token_addresses)
    if not trending_pairs:
        print("No trending pairs available. Using fallback mock data...")
        trending_pairs = [
            {"baseToken": {"symbol": "SHIB", "address": "7xKXtzgLKyTWBEeXUdzeQ7YpTsV9w8TQTz9Y39k9iTQt"},
             "quoteToken": {"symbol": "SOL"}, "volume": {"h24": "1000000"}, "liquidity": {"usd": "500000"}, "priceUsd": "0.0001", "pairAddress": "mock_pair_shib"},
            {"baseToken": {"symbol": "DOGE", "address": "Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB"},
             "quoteToken": {"symbol": "SOL"}, "volume": {"h24": "750000"}, "liquidity": {"usd": "300000"}, "priceUsd": "0.0002", "pairAddress": "mock_pair_doge"}
        ]
    memecoin_data = {}
    for pair in trending_pairs:
        try:
            token_name = pair["baseToken"]["symbol"]
            volume = float(pair["volume"]["h24"])
            if volume < min_volume:
                continue
            if token_name not in memecoin_data or volume > memecoin_data[token_name]["volume_24h"]:
                liquidity = float(pair["liquidity"]["usd"])
                price_usd = float(pair["priceUsd"])
                contract_address = pair["baseToken"]["address"]
                social_score = get_social_engagement(token_name)
                popularity_score = (volume / 1000) + (liquidity / 1000) + social_score
                memecoin_data[token_name] = {
                    "name": token_name,
                    "volume_24h": volume,
                    "liquidity_usd": liquidity,
                    "price_usd": price_usd,
                    "contract_address": contract_address,
                    "social_score": social_score,
                    "popularity_score": popularity_score,
                    "pair_address": pair["pairAddress"]
                }
        except Exception as e:
            print(f"Error processing pair {pair.get('baseToken', {}).get('symbol', 'Unknown')}: {e}")
            continue
    ranked = sorted(memecoin_data.values(), key=lambda x: x["popularity_score"], reverse=True)[:top_n]
    if not ranked:
        print("No memecoins met the minimum volume threshold.")
    return ranked

# Test it
top_coins = rank_memecoins()
for coin in top_coins:
    print(f"{coin['name']}: Volume ${coin['volume_24h']:,.2f}, Popularity {coin['popularity_score']:.2f}")

Token EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm: Status Code 200
Raw Response: {"schemaVersion":"1.0.0","pairs":[{"chainId":"solana","dexId":"raydium","url":"https://dexscreener.com/solana/ep2ib6dydeeqd8mfe2ezhcxx3kp3k2elkkirfpm5eymx","pairAddress":"EP2ib6dYdEeqD8MfE2ezHCxX3kP3K2eLKkirfPm5eyMx","baseToken":{"address":"EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm","name":"dogwifhat","symbol":"$WIF"},"quoteToken":{"address":"So11111111111111111111111111111111111111112","name":"Wrapped SOL","symbol":"SOL"},"priceNative":"0.004222","priceUsd":"0.5586","txns":{"m5":{"buys":11,"
Found 30 Solana pairs for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
Token 5vYksbMzMgKTPbcbyvjo4vA7nkpG168eDcntdjuHjvkw: Status Code 200
Raw Response: {"schemaVersion":"1.0.0","pairs":null}
No 'pairs' key in response for 5vYksbMzMgKTPbcbyvjo4vA7nkpG168eDcntdjuHjvkw
Token 4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R: Status Code 200
Raw Response: {"schemaVersion":"1.0.0","pairs":[{"chainId":"solana","dexId":"rayd

In [19]:
def scrape_tweetscout(handle):
    url = f"https://app.tweetscout.io/account/{handle.lstrip('@')}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        score_elem = soup.find("div", class_=re.compile("score", re.I))
        score = float(score_elem.text.strip()) if score_elem else 0.0
        followers_elem = soup.find("div", text=re.compile("followers", re.I))
        followers = 0
        if followers_elem:
            followers_text = followers_elem.find_next("span").text.strip()
            if "K" in followers_text:
                followers = float(followers_text.replace("K", "")) * 1000
            elif "M" in followers_text:
                followers = float(followers_text.replace("M", "")) * 1000000
            else:
                followers = int(followers_text.replace(",", ""))
        return {"handle": handle, "tweetscout_score": score, "followers": followers}
    except Exception as e:
        print(f"TweetScout error for {handle}: {e}")
        return None

def assess_audience_quality(data):
    if not data:
        return {"quality_score": 0, "quality_label": "Failed"}
    score = data["tweetscout_score"] * 0.7 + (data["followers"] / 1000) * 0.3
    quality = "High" if score >= 500 else "Medium" if score >= 300 else "Low"
    return {"quality_score": score, "quality_label": quality}

In [10]:
def scrape_rugcheck(contract_address):
    url = f"https://rugcheck.xyz/tokens/{contract_address}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        risk_section = soup.find("div", text=re.compile("Risk Score", re.I))
        is_good = False
        if risk_section:
            risk_text = risk_section.find_next("span").text.strip()
            is_good = "Low" in risk_text or "Good" in risk_text
        holders_section = soup.find("div", text=re.compile("Top Holders", re.I))
        holders_data = {}
        if holders_section:
            holders_list = holders_section.find_next("ul")
            if holders_list:
                for li in holders_list.find_all("li")[:5]:
                    holder_text = li.text.strip()
                    match = re.search(r"(\d+\.\d+)%", holder_text)
                    if match:
                        percentage = float(match.group(1))
                        holders_data[holder_text.split()[0]] = percentage
        return {"contract_address": contract_address, "is_good": is_good, "top_holders": holders_data}
    except Exception as e:
        print(f"Rugcheck error for {contract_address}: {e}")
        return {"is_good": False, "top_holders": {}}

def validate_token(data, max_holder_percentage=10.0):
    if not data["is_good"]:
        return False
    for percentage in data["top_holders"].values():
        if percentage > max_holder_percentage:
            return False
    return True

In [84]:
def get_token_price(token_address):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    try:
        response = requests.get(url)
        data = response.json()
        if data["pairs"]:
            return float(data["pairs"][0]["priceUsd"])
        return 0.0
    except Exception:
        return 0.0

def build_swap_instruction(pair_address, amount_in_lamports, slippage=0.05):
    # Simplified Raydium swap - requires pool keys
    # Replace with real pool data from Raydium for live trading
    # Example pool: SOL/USDC (adjust for memecoins)
    pool_address = Pubkey.from_string(pair_address)  # Pair address from Dexscreener
    token_account = Pubkey.from_string("YOUR_TOKEN_ACCOUNT_HERE")  # Your wallet’s token account
    swap_data = bytearray([2])  # Swap instruction (simplified—needs real data)

    accounts = [
        AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
        AccountMeta(pubkey=pool_address, is_signer=False, is_writable=True),
        AccountMeta(pubkey=WALLET_PUBKEY, is_signer=True, is_writable=True),
        AccountMeta(pubkey=token_account, is_signer=False, is_writable=True),
        AccountMeta(pubkey=SYSTEM_PROGRAM_ID, is_signer=False, is_writable=False),
    ]
    return Instruction(
        program_id=RAYDIUM_PROGRAM_ID,
        accounts=accounts,
        data=swap_data
    )

def send_transaction(instructions, priority_fee_micro_lamports=1000):
    tx = Transaction()
    tx.add(set_compute_unit_price(priority_fee_micro_lamports))
    for instr in instructions:
        tx.add(instr)
    try:
        response = sol_client.send_transaction(tx, wallet)
        return response["result"]
    except Exception as e:
        print(f"Transaction failed: {e}")
        return None

def buy_token(pair_address, amount_sol, slippage=0.05, priority_fee=1000):
    amount_lamports = int(amount_sol * 1_000_000_000)
    swap_instruction = build_swap_instruction(pair_address, amount_lamports, slippage)
    tx_hash = send_transaction([swap_instruction], priority_fee)
    if tx_hash:
        print(f"Buy successful: {tx_hash}")
        return tx_hash
    return None

def sell_token(pair_address, amount_token, slippage=0.05, priority_fee=1000):
    amount_lamports = int(amount_token * 1_000_000_000)  # Adjust decimals per token
    swap_instruction = build_swap_instruction(pair_address, amount_lamports, slippage)
    tx_hash = send_transaction([swap_instruction], priority_fee)
    if tx_hash:
        print(f"Sell successful: {tx_hash}")
        return tx_hash
    return None

def monitor_and_take_profit(pair_address, buy_price, take_profit_percent=20.0, amount_token=1.0):
    target_price = buy_price * (1 + take_profit_percent / 100)
    print(f"Monitoring {pair_address} - Target: ${target_price:.6f}")
    for _ in range(60):
        current_price = get_token_price(pair_address)
        print(f"Current: ${current_price:.6f}")
        if current_price >= target_price:
            sell_token(pair_address, amount_token)
            break
        elif current_price <= buy_price * 0.9:
            sell_token(pair_address, amount_token)
            print("Stop-loss triggered.")
            break
        time.sleep(10)

In [21]:
def run_trading_cycle():
    print(f"Starting Trading Cycle at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    top_memecoins = rank_memecoins()  # From Cell 3
    if not top_memecoins:
        print("No memecoins found.")
        return

    for coin in top_memecoins:
        print(f"\nEvaluating {coin['name']} ({coin['contract_address']})")
        
        # Cell 4: TweetScout audience quality (optional)
        x_handle = input(f"Enter X handle for {coin['name']} (or press Enter to skip): ").strip()
        if x_handle:
            audience_data = scrape_tweetscout(x_handle)
            audience_quality = assess_audience_quality(audience_data)
            print(f"Audience Quality: {audience_quality['quality_label']} (Score: {audience_quality['quality_score']:.2f})")
            if audience_quality["quality_label"] == "Low":
                print("Skipping due to low audience quality.")
                continue
        
        # Cell 5: Rugcheck validation
        rugcheck_data = scrape_rugcheck(coin["contract_address"])
        if not validate_token(rugcheck_data):
            print(f"Token failed Rugcheck validation: {rugcheck_data}")
            continue
        
        # Cell 6: Trading
        print("Token passed all checks. Initiating trade...")
        amount_sol = 0.1  # Small trade for testing
        buy_tx = buy_token(coin["pair_address"], coin["contract_address"], amount_sol)
        if buy_tx:
            buy_price = get_token_price(coin["contract_address"])
            if buy_price > 0:
                monitor_and_take_profit(coin["pair_address"], coin["contract_address"], buy_price, amount_token=amount_sol / buy_price)
            else:
                print("Couldn’t fetch price. Holding position.")
        else:
            print("Buy failed.")

# Run the bot
run_trading_cycle()

Starting Trading Cycle at 2025-02-26 21:31:52
Token EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm: Status Code 200
Raw Response: {"schemaVersion":"1.0.0","pairs":[{"chainId":"solana","dexId":"raydium","url":"https://dexscreener.com/solana/ep2ib6dydeeqd8mfe2ezhcxx3kp3k2elkkirfpm5eymx","pairAddress":"EP2ib6dYdEeqD8MfE2ezHCxX3kP3K2eLKkirfPm5eyMx","baseToken":{"address":"EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm","name":"dogwifhat","symbol":"$WIF"},"quoteToken":{"address":"So11111111111111111111111111111111111111112","name":"Wrapped SOL","symbol":"SOL"},"priceNative":"0.004187","priceUsd":"0.5620","txns":{"m5":{"buys":63,"
Found 30 Solana pairs for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
Token 5vYksbMzMgKTPbcbyvjo4vA7nkpG168eDcntdjuHjvkw: Status Code 200
Raw Response: {"schemaVersion":"1.0.0","pairs":null}
No 'pairs' key in response for 5vYksbMzMgKTPbcbyvjo4vA7nkpG168eDcntdjuHjvkw
Token 4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R: Status Code 200
Raw Response: {"schemaVersion":"1.0

Enter X handle for RAY (or press Enter to skip):  kingofrimo


C:\Users\user\AppData\Local\Temp\ipykernel_11392\2976890821.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  followers_elem = soup.find("div", text=re.compile("followers", re.I))


Audience Quality: Low (Score: 0.00)
Skipping due to low audience quality.

Evaluating $WIF (EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm)


Enter X handle for $WIF (or press Enter to skip):  kingofrimo


Audience Quality: Low (Score: 0.00)
Skipping due to low audience quality.
